# Versionamento

V2: 
<ul>
    <li>Inclusão da variável categórica x28 que está relacionada com o tipo de papel em produção</li>
    <li>Feature Selection</li>
    <li>Feature Scalling</li>
</ul>

# O problema

Paper manufacturing can be viewed as a continuous rolling process. During this process, sometimes the paper breaks. If a break happens, the entire process is stopped, the reel is taken out, any found problem is fixed, and the production is resumed. The resumption can take more than an hour.

The cost of this lost production time is significant for a mill. Even a 5% reduction in the break events will give a significant cost saving for a mill. The objective of the given problem is to predict such breaks in advance (early prediction) and identify the potential cause(s) to prevent the break. 

To build such a prediction model, we will use the data collected from the network of sensors in a mill.

This is a multivariate time series data with break as the response (a
binary variable).
The provided data has,
<ul>
    <li>18,398 records.</li>
    <li>Columns:</li>
        <ul>
        <li>time: the timestamp of the row</li>
        <li>y: the binary response variable. There are only 124 rows with y = 1, rest are y = 0.</li>
        <li>x1-x61: predictor variables. All the predictors are continuous variables, except x28 and x61. x61 is a binary variable, and x28 is a
        categorical variable. All the predictors are centered. Besides, the predictors are a mixture of raw materials and process variables. Their descriptions are omitted for data anonymity.
            Several sensors are placed in different parts of the machine along its length
and breadth. These sensors measure both raw materials (e.g. amount of pulp
fiber, chemicals, etc.) and process variables (e.g. blade type, couch vacuum,
rotor speed, etc.).</li>
        </ul>
</ul>

# Setup

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from sklearn.metrics import confusion_matrix, recall_score, precision_score
import datetime

# Load Data

In [ ]:
pwbdatafile = '..\\data\\processminer-rare-event-mts-data.csv'
pwbds = pd.read_csv(pwbdatafile)

In [ ]:
pwbds.head()

In [ ]:
pwbds.shape

In [ ]:
pwbds.describe()

In [ ]:
pwbds.info()

# Exploratory Analysis

In [ ]:
def displayCategorical(yvar):   
    fig, ax = plt.subplots()
    plt.rcParams['font.sans-serif'] = 'Arial'
    plt.rcParams['font.family'] = 'sans-serif'
    plt.rcParams['text.color'] = '#909090'
    plt.rcParams['axes.labelcolor']= '#909090'
    plt.rcParams['xtick.color'] = '#909090'
    plt.rcParams['ytick.color'] = '#909090'
    plt.rcParams['font.size']=12
    labels = yvar.value_counts().keys()
    percentages = list (map(lambda x:round(x*100,2),yvar.value_counts().values/pwbds.shape[0]))
    ax.pie(percentages, labels=labels,  
           autopct='%1.0f%%', 
           shadow=False, startangle=0,   
           pctdistance=1.2,labeldistance=1.4)
    ax.axis('equal')
    ax.set_title("Distribution")
    ax.legend(frameon=False, bbox_to_anchor=(1.5,0.8))

The change in the level of the categorical variable, x28, may be more important than its actual value. This variable is related to the type of paper produced at that time. For this prediction model, it might be more important to capture any change in the paper type instead of the actual type of the paper.
May consider adding a feature capturing the change in x28, e.g. x28t − x28t−1.

In [ ]:
displayCategorical(pwbds['x28'])

In [ ]:
displayCategorical(pwbds['x61'])

In [ ]:
# A distribuição de x61 indica que podemos descartá-la no processo
pwbds.drop('x61',axis='columns', inplace=True)

In [ ]:
# Now, the target
pwbds.y.value_counts()

123 registros de quebra da teia

In [ ]:
train_ind = int(0.8 * pwbds.shape[0])
y_test = pwbds.y[train_ind:]
np.unique(y_test,return_counts=True)

Haverá 34 eventos de quebra no conjunto de teste

In [ ]:
#pwbds['date'] = pwbds['time'].str.split(' ').str[0]
#pwbds['time'] = pwbds['time'].str.split(' ').str[1]
pwbds['time']= pd.to_datetime(pwbds['time'])

## Check total downtime

In [ ]:
def calcDownTime(ds):
    dstmp = ds.copy()
    dstmp["next_measure"] = dstmp["time"].shift(-1)
    dstmp['downtime'] = dstmp['next_measure'] - dstmp['time']
    downtimes = dstmp.loc[dstmp['y']==1][['time', 'downtime']].copy()
    return (dstmp.groupby(['y'])['downtime'].agg('sum').iloc[1,],downtimes)

In [ ]:
totalPeriod = pwbds.tail(1)['time'].iloc[0,] - pwbds.head(1)['time'].iloc[0,]
totalDowntime,dtds = calcDownTime(pwbds)
perc = round ((totalDowntime/totalPeriod)*100,2)
print ('Downtime total de:', totalDowntime, 'num período de: ', totalPeriod, 'correspondendo a: ',perc,'%')

## Breaks per day

In [ ]:
dtds['time'] = dtds['time'].astype(str)
dtds['time'] = dtds['time'].str.split(' ').str[0]

In [ ]:
# Determine max_slots
max_slots = dtds.time.value_counts().max()
slot_size = len(dtds.time.value_counts()) 

In [ ]:
dtds['downtime']=dtds['downtime'].dt.total_seconds()/60

In [ ]:
# Monta numero de listas correspondentes à max_slots, de tamanho correspondente ao número de dias
downtime_matrix = np.zeros([max_slots,slot_size])
day_idx=0
reg_idx=0
lista_idx=-1
last_day = dtds['time'].iloc[0,]
for day in dtds['time'].tolist():
    if (day != last_day):
        lista_idx=0
        last_day = day
        day_idx+=1
    else:
        lista_idx+=1
    downtime_matrix[lista_idx,day_idx]=dtds.downtime.iloc[reg_idx,]
    reg_idx+=1

In [ ]:
downtimedf = pd.DataFrame (downtime_matrix.T)
downtimedf['day'] = dtds.time.value_counts().sort_index().index

In [ ]:
downtimedf = downtimedf.set_index('day')

In [ ]:
downtimedf.plot(kind="bar",stacked=True, legend=None,figsize=(20, 10))
plt.title("Downtime Occurencies")
plt.xlabel("Day")
plt.ylabel("Downtime minutes")

## Comportamento de cada feature no tempo

In [ ]:
cols = [
    'y','x1','x2','x3','x4','x5','x6','x7','x8','x9','x10','x11','x12','x13','x14','x15','x16','x17','x18','x19','x20',
    'x21','x22','x23','x24','x25','x26','x27','x29','x30','x31','x32','x33','x34','x35','x36','x37','x38','x39','x40',
    'x41','x42','x43','x44','x45','x46','x47','x48','x49','x50','x51','x52','x53','x54','x55','x56','x57','x58','x59','x60',
    ]
sns.set_theme(style="darkgrid")


for col in cols:
    fig, ax = plt.subplots()
    fig.set_size_inches(18, 2)
#    col_ds = pwbds.groupby('time').agg({col:np.median}).reset_index()
    sns.lineplot(x="time", y=col, data=pwbds, ax=ax)

## Cálculo do Remaining Usefull Lifecycle (RUL)

In [ ]:
# Dates whith break
df_sub = sorted(pwbds[pwbds['y'] == 1]['time'].tolist())

In [ ]:
# variable to store all days
breakSubIdx=0
breakLstIdx=0
breakList = pwbds['y'].tolist()
nextbreak = []
for v1 in pwbds['time'].tolist():
    if((breakList[breakLstIdx] == 1)):
#        print ('1')
        nextbreak.append(v1)
        breakSubIdx = breakSubIdx+1
    else:
#        print('0')
        if (breakSubIdx < len(df_sub)):
            nextbreak.append(df_sub[breakSubIdx])
        else:
            nextbreak.append(df_sub[breakSubIdx-1])
    breakLstIdx = breakLstIdx+1
pwbds['nextbreaktime'] = pd.Series(nextbreak)

In [ ]:
pwbds['RUL']=round((pwbds['nextbreaktime']-pwbds['time']).dt.total_seconds()/60,2)
pwbds.drop(pwbds[pwbds.RUL < 0].index, inplace=True)
pwbds.drop(['time','nextbreaktime'], axis=1, inplace=True)

## Marcação de falha no próximo período (próxima hora)

Using RUL, we can create a label indicating time to failure. We define a boolean (True\False) value for NEXT_H indicating the engine will fail within 60 minutes (RUL  <=60 ). 

We can also define a multiclass MULTI  ∈{0,1,2}  indicating {Healthy, RUL <=60, RUL <=120} minutes.

In [ ]:
pwbds['NEXT_H'] = np.where(pwbds['RUL'] <= 60, 1, 0 )

In [ ]:
pwbds['NEXT_H'].value_counts()

# Feature Selection and Scaling

In [ ]:
# Feature Selection
# List of considered Features
FEATURES = [
    'x1','x2','x3','x4','x5','x6','x7','x8','x9','x10','x11','x13','x14','x15','x17','x18','x19','x20',
    'x21','x22','x24','x26','x27','x28','x29','x30','x32','x33','x34','x35','x36','x37','x38','x39','x40',
    'x41','x42','x43','x44','x45','x46','x47','x48','x49','x50','x54','x55','x56','x57','x60',
    ]

# Create the dataset with features and filter the data to the list of FEATURES
pwbds_filtered = pwbds[FEATURES]

# Print the tail of the dataframe
pwbds_filtered.tail()

In [ ]:
# This Scaler removes the median and scales the data according to the quantile range to normalize the price data 
from sklearn.preprocessing import RobustScaler, MinMaxScaler 

# Get the number of rows in the data
nrows = pwbds_filtered.shape[0]

# Convert the data to numpy values
np_data_unscaled = np.array(pwbds_filtered)
np_data = np.reshape(np_data_unscaled, (nrows, -1))
print(np_data.shape)

# Transform the data by scaling each feature to a range between 0 and 1
scaler = MinMaxScaler()
np_data_scaled = scaler.fit_transform(np_data_unscaled)
pwbds_scaled = pd.DataFrame(
    np_data_scaled,
    columns=FEATURES
)
pwbds_scaled['RUL'] = pwbds['RUL']

# Shaping & Splitting

In [ ]:
def lstm_data_transform(x_data, y_data, num_steps=5):
    """ Changes data to the format for LSTM training for sliding window approach """
    # Prepare the list for the transformed data
    X, y = list(), list()
    # Loop of the entire data set
    for i in range(x_data.shape[0]):
        # compute a new (sliding window) index
        end_ix = i + num_steps
        # if index is larger than the size of the dataset, we stop
        if end_ix >= x_data.shape[0]:
            break
        # Get a sequence of data for x
        seq_X = x_data[i:end_ix]
        # Get only the last element of the sequency for y
        seq_y = y_data[end_ix]
        # Append the list with sequencies
        X.append(seq_X)
        y.append(seq_y)
    # Make final arrays
    x_array = np.array(X)
    y_array = np.array(y)
    return x_array, y_array

## Dataset for Model 1

In [ ]:
pwbds_m1 = pwbds_scaled.copy()
#pwbds_m1.drop('NEXT_H',axis='columns', inplace=True)
yds_m1 = pwbds_m1.pop('RUL')

In [ ]:
num_steps = pwbds_m1.shape[1]
x_new, y_new = lstm_data_transform(pwbds_m1, yds_m1, num_steps=num_steps)
print ("The new shape of x is", x_new.shape)

In [ ]:
train_ind = int(0.8 * pwbds.shape[0])
x_train = x_new[:train_ind]
y_train = y_new[:train_ind]
x_test = x_new[train_ind:]
y_test = y_new[train_ind:]

# Modeling 1: Regression - Predict RUL

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
model_1 = keras.Sequential()
model_1.add(layers.LSTM(100, activation='tanh', input_shape=(num_steps, x_train.shape[2]), 
               return_sequences=True))
# Plus a 20% dropout rate
model_1.add(layers.Dropout(0.2))

# The second layer
model_1.add(layers.LSTM(
          units=50,
          return_sequences=False))

# Plus a 20% dropout rate
model_1.add(layers.Dropout(0.2))
model_1.add(layers.Dense(units=50, activation='relu'))
model_1.add(layers.Dense(units=1, activation='linear'))
adam = keras.optimizers.Adam(lr=0.0001)
model_1.compile(optimizer=adam, loss='mse')

In [ ]:
# Verify the architecture 
print(model_1.summary())

In [ ]:
# EarlyStopping Callback
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

history_m1 = model_1.fit(
    x_train, 
    y_train, 
    epochs=40,
    callbacks=[callback]
#    batch_size=200, 
#    validation_split=0.10 # Use 10% of data to evaluate the loss. (val_loss)
)

In [ ]:
# Plot History
pd.DataFrame(history_m1.history).plot()
plt.ylabel('loss')
plt.xlabel('epochs')

In [ ]:
test_predict = model_1.predict(x_test)

In [ ]:
figure(figsize=(15, 6), dpi=80)
plt.plot(y_test,'r+')
plt.plot(test_predict,'b+')

In [ ]:
import shap  # package used to calculate Shap values

# use Deep SHAP to explain test set predictions
#deep_explainer = shap.DeepExplainer(model_1.predict_proba, x_train)
#deep_shap_values = deep_explainer.shap_values(x_test)
#shap.force_plot(deep_explainer.expected_value[1], deep_shap_values[1], x_test)

## Truth

### Quantas quebras realmente ocorreram?

In [ ]:
np.unique(y_test==0,return_counts=True)

### Quando ocorreram?

In [ ]:
break_moments= np.where(y_test == 0)

In [ ]:
list(break_moments)

## Previsões

### Quantas indicações abaixo de x minutos? 

In [ ]:
np.unique(test_predict < 60 ,return_counts=True)

### Quando ocorreram?

In [ ]:
np.where(test_predict < 60)[0]

## Acertos
### Quantos e quando?

In [ ]:
when_pred = np.where(test_predict < 240)[0].tolist()
when_true = np.where(y_test == 0)[0].tolist()

In [ ]:
commonalities = set(when_pred) - (set(when_pred) - set(when_true))

In [ ]:
commonalities

# Modeling 2: Binary Classification - Break in the next hour

## Dataset for model 2

In [ ]:
pwbds_m2 = pwbds
yds_m2 = pwbds_m2.pop('NEXT_H')

In [ ]:
num_steps = pwbds_m2.shape[1]
x_new, y_new = lstm_data_transform(pwbds_m2, yds_m2, num_steps=num_steps)
print ("The new shape of x is", x_new.shape)

In [ ]:
train_ind = int(0.8 * pwbds.shape[0])
x_train = x_new[:train_ind]
y_train = y_new[:train_ind]
x_test = x_new[train_ind:]
y_test = y_new[train_ind:]

# Model 2

In [ ]:
model_2 = keras.Sequential()
model_2.add(layers.LSTM(100, activation='tanh', input_shape=(num_steps, x_train.shape[2]), 
               return_sequences=True))
# Plus a 20% dropout rate
model_2.add(layers.Dropout(0.2))

# The second layer
model_2.add(layers.LSTM(
          units=50,
          return_sequences=False))

# Plus a 20% dropout rate
model_2.add(layers.Dropout(0.2))
model_2.add(layers.Dense(units=50, activation='relu'))
model_2.add(layers.Dense(units=1, activation='sigmoid'))
adam = keras.optimizers.Adam(lr=0.0001)
model_2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Verify the architecture 
print(model_2.summary())

In [ ]:
history_m2 = model_2.fit(
    x_train, 
    y_train, 
    epochs=20,
    callbacks=[callback]
#    batch_size=200, # 
#    validation_split=0.10 # Use 10% of data to evaluate the loss. (val_loss)
)

In [ ]:
# Plot History
pd.DataFrame(history_m2.history).plot()
plt.ylabel('loss')
plt.xlabel('epochs')

In [ ]:
#scores_2 = model_2.evaluate(x_test, y_test, verbose=1, batch_size=200)
scores_2 = model_2.evaluate(x_test, y_test, verbose=1)
print('Training Accurracy: {}'.format(scores_2[1]))

In [ ]:
# make predictions and compute confusion matrix
#y_pred = model_2.predict_classes(x_test,verbose=1, batch_size=200)
y_pred = model_2.predict_classes(x_test,verbose=1)
y_true = y_test
print('Training Confusion matrix\n- x-axis is true labels.\n- y-axis is predicted labels')
cm = confusion_matrix(y_true, y_pred)
cm

In [ ]:
# compute precision and recall
precision_test = precision_score(y_true, y_pred)
recall_test = recall_score(y_true, y_pred)
f1_test = 2 * (precision_test * recall_test) / (precision_test + recall_test)
print( 'Test Precision: ', precision_test, '\n', 'Test Recall: ', recall_test, '\n', 'Test F1 Score:', f1_test)

In [ ]:
y_pred

In [ ]:
cm = confusion_matrix(y_true, y_pred)

In [ ]:
cmtx = pd.DataFrame(
    confusion_matrix(y_true, y_pred),
    index=['true:0', 'true:1'], 
    columns=['pred:0', 'pred:1']
)
print(cmtx)